In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy import stats
from sklearn import decomposition, linear_model, model_selection, ensemble
import warnings
from rpy2 import robjects
import rpy2.robjects.numpy2ri
import multiprocessing
import glmnet_python
from glmnet import glmnet
from glmnetPredict import glmnetPredict
from cvglmnet import cvglmnet
from cvglmnetPredict import cvglmnetPredict
import pickle
import scipy
# tqdm is not a necessary dependency
#from tqdm import tqdm_notebook as tqdm

robjects.r.library("irr")
robjects.r.library("psych")
pass

# Load Data

In [ ]:
hcp_behavioral_data = pd.read_csv('../data/unrestricted_HCP_behavioral_data.csv')  # load unrestricted HCP behavioral data, available for download at http://db.humanconnectome.org
hcp_behavioral_data_restricted = pd.read_csv('../data/restricted_HCP_behavioral_data.csv')  # load restricted HCP behavioral data, available for download at http://db.humanconnectome.org
hcp_factors = pd.read_csv('../data/factors.csv')
hcp_behavioral_data = hcp_behavioral_data.merge(hcp_factors, on='Subject', how='left')
hcp_behavioral_data = hcp_behavioral_data.merge(hcp_behavioral_data_restricted[['Subject', 'ASR_Intn_T', 'ASR_Extn_T', 'ASR_Attn_Pct']], on='Subject', how='left')

In [ ]:
sess1_matrix = pickle.load(open('../data/sess1_matrix.pickle', 'rb'))
sess2_matrix = pickle.load(open('../data/sess2_matrix.pickle', 'rb'))
folds = pickle.load(open('../data/folds.pickle', 'rb'))

In [ ]:
ALL_PHEN = ['GenExec','ProcSpeed','PMAT24_A_CR','ASR_Extn_T',
            'ASR_Intn_T','ASR_Attn_Pct','NEOFAC_O','NEOFAC_C','NEOFAC_E',
            'NEOFAC_A','NEOFAC_N',
            'DDisc_AUC_40K','ProcSpeed_Unadj','PicSeq_Unadj',
            'CardSort_Unadj','Flanker_Unadj','ListSort_Unadj',
            'ReadEng_Unadj','PicVocab_Unadj','SCPT_SEN',
            'SCPT_SPEC','IWRD_TOT','VSPLOT_TC',
            'MMSE_Score','PSQI_Score','Endurance_Unadj','GaitSpeed_Comp',
            'Dexterity_Unadj','Strength_Unadj','Odor_Unadj',
            'PainInterf_Tscore','Taste_Unadj','Mars_Final',
            'Emotion_Task_Face_Acc','Language_Task_Math_Avg_Difficulty_Level',
            'Language_Task_Story_Avg_Difficulty_Level','Social_Task_Perc_Random',
            'Social_Task_Perc_TOM','WM_Task_Acc','ER40_CR','ER40ANG',
            'ER40FEAR','ER40NOE','ER40SAD','AngAffect_Unadj',
            'AngHostil_Unadj','AngAggr_Unadj','FearAffect_Unadj',
            'FearSomat_Unadj','Sadness_Unadj','LifeSatisf_Unadj',
            'MeanPurp_Unadj','PosAffect_Unadj','Friendship_Unadj',
            'Loneliness_Unadj','PercHostil_Unadj','PercReject_Unadj',
            'EmotSupp_Unadj','InstruSupp_Unadj','PercStress_Unadj','SelfEff_Unadj']

cols = ALL_PHEN
cols.append('Subject')
rest_data_df = hcp_behavioral_data[cols]
rest_data_df = rest_data_df.dropna()

# Cross Validation Helper Functions

These functions define the cross validation scheme described in the paper, used to compute model ICCs and Accuracies

In [ ]:
def print_message(msg_id, fold_i, sess_i):
    fold_i = int(fold_i)
    if msg_id == 0:
        print(f'CV Fold: {fold_i+1:<10} Preprocessing session {sess_i} data...')
    elif msg_id == 1:
        print(f'CV Fold: {fold_i+1:<10} Fitting session {sess_i} model...')
    elif msg_id == 2:
        print(f'CV Fold: {fold_i+1:<10} Predicting session {sess_i} model...')
    elif msg_id == 3:
        print(f'CV Fold: {fold_i+1:<10} Scoring session {sess_i} model...')
    elif msg_id == 4:
        print('='*55)
    else:
        pass


# for each fold,
# train on session 1
#   predict on session 1 and 2
#   compute ICC of predictions
# train on session 2
#   predict on session 2 and 1
#   compute ICC of predictions
# final results is average of 10 ICCs

def cross_validate(sess1_X, sess2_X, Y, 
                   model_trainer, model_predictor, model_scorer, 
                   center_X, scale_X, center_Y, scale_Y, 
                   folds, enable_tqdm=False, verbose=True, tqdm_desc=''):
    if enable_tqdm: print()
    folds_scores = []
    sess1_folds_preds = []
    sess2_folds_preds = []
    models = []
    fold_id_iter = tqdm(np.arange(folds.min(), folds.max()+1), desc=tqdm_desc, ncols='50%') if enable_tqdm else np.arange(folds.min(), folds.max()+1)
    for fold_id in fold_id_iter:
        train_idxs = np.argwhere(folds != fold_id).flatten()
        test_idxs = np.argwhere(folds == fold_id).flatten()
        
        train_Y_mu = Y[train_idxs].mean() if center_Y else 0
        train_Y_sd = Y[train_idxs].std() if scale_Y else 1
        Y_std = (Y - train_Y_mu) / train_Y_sd
        
        # model for session 1
        if verbose: print_message(0, fold_id, 1)
        train_X_mu = sess1_X[train_idxs, :].mean(0) if center_X else 0 
        train_X_sd = sess1_X[train_idxs, :].std(0) if scale_X else 1
        sess1_X_std = (sess1_X - train_X_mu) / train_X_sd
        sess2_X_std = (sess2_X - train_X_mu) / train_X_sd
        if verbose: print_message(1, fold_id, 1)
        sess1_model = model_trainer(sess1_X_std[train_idxs, :], Y_std[train_idxs])
        if verbose: print_message(2, fold_id, 1)
        sess1_preds = model_predictor(sess1_X_std[test_idxs, :], sess1_model)
        sess2_preds = model_predictor(sess2_X_std[test_idxs, :], sess1_model)
        if verbose: print_message(3, fold_id, 1)
        sess1_model_scores = model_scorer(sess1_preds, sess2_preds, Y_std[test_idxs])
        sess1_folds_preds.append((sess1_preds, sess2_preds, Y_std[test_idxs]))
        
        # model for session 2
        if verbose: print_message(0, fold_id, 2)
        train_X_mu = sess2_X[train_idxs, :].mean(0) if center_X else 0 
        train_X_sd = sess2_X[train_idxs, :].std(0) if scale_X else 1
        sess1_X_std = (sess1_X - train_X_mu) / train_X_sd
        sess2_X_std = (sess2_X - train_X_mu) / train_X_sd
        if verbose: print_message(1, fold_id, 2)
        sess2_model = model_trainer(sess2_X_std[train_idxs, :], Y_std[train_idxs])
        if verbose: print_message(2, fold_id, 2)
        sess1_preds = model_predictor(sess1_X_std[test_idxs, :], sess2_model)
        sess2_preds = model_predictor(sess2_X_std[test_idxs, :], sess2_model)
        if verbose: print_message(3, fold_id, 2)
        sess2_model_scores = model_scorer(sess1_preds, sess2_preds, Y_std[test_idxs])
        sess2_folds_preds.append((sess1_preds, sess2_preds, Y_std[test_idxs]))
        
        folds_scores.append((sess1_model_scores, sess2_model_scores)) 
        models.append((sess1_model, sess2_model))
        if verbose: print_message(4, fold_id, np.float('nan'))
            
    return folds_scores, sess1_folds_preds, sess2_folds_preds, []  # can return models as last value, skipped to save disk-space


def default_scorer(preds1, preds2, trueY):
    with warnings.catch_warnings():
        # we expect RRuntimeWarning from ICC calling lmer for random effects modelling
        warnings.simplefilter("ignore")
        preds_mat2D = np.stack((preds1, preds2)).T
        rpy2.robjects.numpy2ri.activate()
        icc_2_1 = robjects.r.ICC(preds_mat2D)[0][1][1]
    # score per fold is 
    # (ICC, pred_corr, sess1_acc, sess2_acc)
    return icc_2_1, stats.pearsonr(preds1, preds2)[0], stats.pearsonr(preds1, trueY)[0], stats.pearsonr(preds2, trueY)[0]


def multi_cross_validate(phen_names, cross_validate_fn, sess1_X=sess1_matrix, sess2_X=sess2_matrix, n_procs=1):    
    if __name__ == '__main__':
        if n_procs > 1:
            pool = multiprocessing.Pool(n_procs)
            results = pool.starmap(cross_validate_fn, [(sess1_X, sess2_X, rest_data_df[[phen]].values.flatten(), f'({int(phen_idx/4)}) {phen}') for phen_idx, phen in enumerate(phen_names)])
            pool.close()
            return dict(zip(phen_names, results))
        else:
            results = []
            for phen_idx, phen in enumerate(phen_names):
                results.append(cross_validate_fn(sess1_X, sess2_X, rest_data_df[[phen]].values.flatten(), f'({int(phen_idx)}) {phen}'))
            return dict(zip(phen_names, results))
    else:
        raise Exception()

# BBS-75

In [ ]:
def BBS_trainer(X, Y, n_components=75):
        # fit pca
        pca_model = decomposition.PCA(n_components=n_components, random_state=42).fit(X)
        # dimension reduce
        X_transformed = pca_model.transform(X)
        # fit OLS model
        lr_model = linear_model.LinearRegression(fit_intercept=True, normalize=False)
        lr_model.fit(X_transformed, Y)
        return pca_model, lr_model
    
    
def BBS_predictor(X, model):
    pca_model, lr_model = model
    X_transformed = pca_model.transform(X)
    return lr_model.predict(X_transformed)

def BBS_cross_validate_fn(sess1_X, sess2_X, Y, tqdm_desc): 
    return cross_validate(sess1_X, sess2_X, Y,
                          BBS_trainer, BBS_predictor, default_scorer, 
                          center_X=True, scale_X=False, center_Y=False, scale_Y=False, 
                          folds=folds, enable_tqdm=False, verbose=False, tqdm_desc=tqdm_desc)

BBS_results = multi_cross_validate(ALL_PHEN, BBS_cross_validate_fn, n_procs=1)

In [ ]:
pickle.dump(BBS_results, open('./saved_models/BBS75_results.pickle', 'wb'))

# BBS-CV

In [ ]:
class BBSEstimator:
    def __init__(self, n_components):
        self.n_components = n_components
        self.pca_model = None
        self.linear_model = None

    def fit(self, X, y):
        pca_model = decomposition.PCA(n_components=self.n_components, random_state=42).fit(X)
        X_transformed = pca_model.transform(X)
        lr_model = linear_model.LinearRegression(fit_intercept=True, normalize=False).fit(X_transformed, y)
        self.pca_model = pca_model
        self.linear_model = lr_model
        
    def get_params(self, deep = False):
        return {'n_components': self.n_components}
                #'pca_model': self.pca_model,
                #'linear_model': self.linear_model}


def BBS_scorer(estimator, X, y):
    pca_model = estimator.pca_model
    linear_model = estimator.linear_model
    return stats.pearsonr(linear_model.predict(pca_model.transform(X)), y)[0]


def BBSCV_trainer(X, Y, n_components=[25, 75, 150, 250]):
    comp_scores = []
    for n_comp in n_components:
        scores = model_selection.cross_val_score(BBSEstimator(n_components=n_comp), X, Y, cv=5, scoring=BBS_scorer)
        comp_scores.append(np.nanmean(scores))
    
    optimal_n_components_idx = comp_scores.index(max(comp_scores))
    opt_n_components = n_components[optimal_n_components_idx]
    # fit pca
    pca_model = decomposition.PCA(n_components=opt_n_components, random_state=42).fit(X)
    # fit OLS model
    lr_model = linear_model.LinearRegression(fit_intercept=True, normalize=False)
    lr_model.fit(pca_model.transform(X), Y)
    return pca_model, lr_model
    
    
def BBSCV_predictor(X, model):
    pca_model, lr_model = model
    return lr_model.predict(pca_model.transform(X))

def BBSCV_cross_validate_fn(sess1_X, sess2_X, Y, tqdm_desc): 
    return cross_validate(sess1_X, sess2_X, Y,
                          BBSCV_trainer, BBSCV_predictor, default_scorer, 
                          center_X=True, scale_X=False, center_Y=False, scale_Y=False, 
                          folds=folds, enable_tqdm=True, verbose=False, tqdm_desc=tqdm_desc)

BBSCV_results = multi_cross_validate(ALL_PHEN, BBSCV_cross_validate_fn, n_procs=1)

In [ ]:
pickle.dump(BBSCV_results, open('./saved_models/BBSCV_results.pickle', 'wb'))

# Lasso

In [ ]:
def LASSO_trainer(X, Y, n_cv=5):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ridge_model = cvglmnet(alpha=1, nlambda=100, x=X, y=Y, ptype = 'mse', nfolds=n_cv, parallel=True, standardize=False)
    return ridge_model
    
def LASSO_predictor(X, model):
    return cvglmnetPredict(model, newx=X, s='lambda_min').flatten()

def LASSO_cross_validate_fn(sess1_X, sess2_X, Y, tqdm_desc): 
    return cross_validate(sess1_X, sess2_X, Y,
                          LASSO_trainer, LASSO_predictor, default_scorer, 
                          center_X=False, scale_X=False, center_Y=False, scale_Y=False, 
                          folds=folds, enable_tqdm=True, verbose=False, tqdm_desc=tqdm_desc)

Lasso_results = multi_cross_validate(ALL_PHEN, LASSO_cross_validate_fn, n_procs=1)

In [ ]:
pickle.dump(Lasso_results, open('./saved_models/Lasso_results.pickle', 'wb'))

# Ridge

In [ ]:
def Ridge_trainer(X, Y, n_cv=5):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ridge_model = cvglmnet(alpha=0, nlambda=100, x=X, y=Y, ptype = 'mse', nfolds=n_cv, parallel=True, standardize=False)
    return ridge_model
    
def Ridge_predictor(X, model):
    return cvglmnetPredict(model, newx=X, s='lambda_min').flatten()

def Ridge_cross_validate_fn(sess1_X, sess2_X, Y, tqdm_desc): 
    return cross_validate(sess1_X, sess2_X, Y,
                          Ridge_trainer, Ridge_predictor, default_scorer, 
                          center_X=False, scale_X=False, center_Y=False, scale_Y=False, 
                          folds=folds, enable_tqdm=True, verbose=False, tqdm_desc=tqdm_desc)

Ridge_results = multi_cross_validate(ALL_PHEN, Ridge_cross_validate_fn, n_procs=1)

In [ ]:
pickle.dump(Ridge_results, open('./saved_models/Ridge_results.pickle', 'wb'))

# Elastic Net

In [ ]:
def EnetCV_trainer(X, Y, alphas=[0, 0.1, 0.325, 0.55, 0.775, 1]):
    with warnings.catch_warnings():
        comp_scores = []
        for alpha in alphas:
            model = cvglmnet(alpha=alpha, nlambda=100, x=X, y=Y, ptype = 'mse', nfolds=5, parallel=True, standardize=False)
            preds = cvglmnetPredict(model, newx=X, s='lambda_min').flatten()
            comp_scores.append(stats.pearsonr(preds, Y)[0])


        optimal_n_components_idx = comp_scores.index(max(comp_scores))
        opt_alpha = alphas[optimal_n_components_idx]
        # fit pca
        enet_model = cvglmnet(alpha=opt_alpha, nlambda=100, x=X, y=Y, ptype = 'mse', nfolds=5, parallel=True, standardize=False)
    return enet_model
    
    
def EnetCV_predictor(X, model):
    return cvglmnetPredict(model, newx=X, s='lambda_min').flatten()

def EnetCV_cross_validate_fn(sess1_X, sess2_X, Y, tqdm_desc): 
    return cross_validate(sess1_X, sess2_X, Y,
                          EnetCV_trainer, EnetCV_predictor, default_scorer, 
                          center_X=False, scale_X=False, center_Y=False, scale_Y=False, 
                          folds=folds, enable_tqdm=True, verbose=False, tqdm_desc=tqdm_desc)

EnetCV_results = multi_cross_validate(ALL_PHEN, EnetCV_cross_validate_fn, n_procs=1)

In [ ]:
pickle.dump(EnetCV_results, open('./saved_models/ENet_results.pickle', 'wb'))

# CPM Positive

In [ ]:
# CPM Positive
def CPM_trainer(X, Y, p_threshold=0.01, is_pos=True):
    with warnings.catch_warnings():
        # we expect pearsonr to throw PearsonRConstantInputWarning because of contant valued columns in X
        warnings.simplefilter("ignore")
        pheno_corr_p = [stats.pearsonr(X[:, i], Y) for i in range(X.shape[1])]
        X_corrs = np.array([x[0] for x in pheno_corr_p])
        X_pvals = np.array([x[1] for x in pheno_corr_p])
        # create masks for edges below p-threshold and split pos/neg correlations
        keep_edges_pos = (X_corrs > 0) & (X_pvals < p_threshold)
        keep_edges_neg = (X_corrs < 0) & (X_pvals < p_threshold)

    # sum X entries with significant correlations with y
    X_pos_edges_sum = X[:, keep_edges_pos].sum(1)
    X_neg_edges_sum = X[:, keep_edges_neg].sum(1)
    
    X_train = X_pos_edges_sum.reshape(-1, 1) if is_pos else X_neg_edges_sum.reshape(-1, 1)
    keep_edges = keep_edges_pos if is_pos else keep_edges_neg
    model =  linear_model.LinearRegression(fit_intercept=True, normalize=False).fit(X_train, Y)
    return model, keep_edges

    
def CPM_predictor(X, model):
    lr_model, keep_edges = model
    X_edges_sum = X[:, keep_edges].sum(1)
    return lr_model.predict(X_edges_sum.reshape(-1, 1))


def CPM_cross_validate_fn(sess1_X, sess2_X, Y, tqdm_desc): 
    return cross_validate(sess1_X, sess2_X, Y,
                          CPM_trainer, CPM_predictor, default_scorer, 
                          center_X=False, scale_X=False, center_Y=False, scale_Y=False, 
                          folds=folds, enable_tqdm=True, verbose=False, tqdm_desc=tqdm_desc)

CPM_pos_results = multi_cross_validate(ALL_PHEN, CPM_cross_validate_fn, n_procs=1)

In [ ]:
pickle.dump(CPM_pos_results, open('./saved_models/CPMPos_results.pickle', 'wb'))

# CPM Negative

In [ ]:
# CPM Positive
def CPM_trainer(X, Y, p_threshold=0.01, is_pos=False):
    with warnings.catch_warnings():
        # we expect pearsonr to throw PearsonRConstantInputWarning because of contant valued columns in X
        warnings.simplefilter("ignore")
        pheno_corr_p = [stats.pearsonr(X[:, i], Y) for i in range(X.shape[1])]
        X_corrs = np.array([x[0] for x in pheno_corr_p])
        X_pvals = np.array([x[1] for x in pheno_corr_p])
        # create masks for edges below p-threshold and split pos/neg correlations
        keep_edges_pos = (X_corrs > 0) & (X_pvals < p_threshold)
        keep_edges_neg = (X_corrs < 0) & (X_pvals < p_threshold)

    # sum X entries with significant correlations with y
    X_pos_edges_sum = X[:, keep_edges_pos].sum(1)
    X_neg_edges_sum = X[:, keep_edges_neg].sum(1)
    
    X_train = X_pos_edges_sum.reshape(-1, 1) if is_pos else X_neg_edges_sum.reshape(-1, 1)
    keep_edges = keep_edges_pos if is_pos else keep_edges_neg
    model =  linear_model.LinearRegression(fit_intercept=True, normalize=False).fit(X_train, Y)
    return model, keep_edges

    
def CPM_predictor(X, model):
    lr_model, keep_edges = model
    X_edges_sum = X[:, keep_edges].sum(1)
    return lr_model.predict(X_edges_sum.reshape(-1, 1))


def CPM_cross_validate_fn(sess1_X, sess2_X, Y, tqdm_desc): 
    return cross_validate(sess1_X, sess2_X, Y,
                          CPM_trainer, CPM_predictor, default_scorer, 
                          center_X=False, scale_X=False, center_Y=False, scale_Y=False, 
                          folds=folds, enable_tqdm=True, verbose=False, tqdm_desc=tqdm_desc)

CPM_neg_results = multi_cross_validate(ALL_PHEN, CPM_cross_validate_fn, n_procs=1)

In [ ]:
pickle.dump(CPM_neg_results, open('./saved_models/CPMNeg_results.pickle', 'wb'))

# SVR Linear

In [ ]:
class SVREstimator:
    def __init__(self, C, eps):
        self.C = C
        self.eps = eps
        self.svr_model = None

    def fit(self, X, y):
        with warnings.catch_warnings():
            svr_model = svm.LinearSVR(C=self.C, epsilon=self.eps).fit(X, y)
            self.svr_model = svr_model
        
    def get_params(self, deep = False):
        return {'C': self.C,
                'eps': self.eps}


def SVR_scorer(estimator, X, y):
    return stats.pearsonr(estimator.svr_model.predict(X), y)[0]


def SVRCV_trainer(X, Y, c_eps_vals = [(0.1, 0.1), (0.1, 1), (0.1, 10), 
                                      (1, 0.1), (1, 1), (1, 10), 
                                      (10, 0.1), (10, 1), (10, 10)]):
    comp_scores = []
    for C, eps in c_eps_vals:
        scores = model_selection.cross_val_score(SVREstimator(C=C, eps=eps), X, Y, cv=5, scoring=SVR_scorer, n_jobs=5)
        comp_scores.append(np.nanmean(scores))
    
    optimal_n_components_idx = comp_scores.index(max(comp_scores))
    opt_C, opt_eps = c_eps_vals[optimal_n_components_idx]
    svr_model = svm.LinearSVR(C=opt_C, epsilon=opt_eps).fit(X, Y)
    return svr_model
    
    
def SVRCV_predictor(X, model):
    return model.predict(X)

def SVRCV_cross_validate_fn(sess1_X, sess2_X, Y, tqdm_desc): 
    return cross_validate(sess1_X, sess2_X, Y,
                          SVRCV_trainer, SVRCV_predictor, default_scorer, 
                          center_X=False, scale_X=False, center_Y=True, scale_Y=True, 
                          folds=folds, enable_tqdm=False, verbose=False, tqdm_desc=tqdm_desc)

SVRCV_results = multi_cross_validate(ALL_PHEN, SVRCV_cross_validate_fn, n_procs=1)

In [ ]:
pickle.dump(SVRCV_results, open('./saved_models/SVRLin_results.pickle', 'wb'))

# SVR Non Linear

In [ ]:
class SVREstimator:
    def __init__(self, C, eps):
        self.C = C
        self.eps = eps
        self.svr_model = None

    def fit(self, X, y):
        with warnings.catch_warnings():
            svr_model = svm.SVR(kernel='rbf', C=self.C, epsilon=self.eps, cache_size=10000, max_iter=1000).fit(X, y)
            self.svr_model = svr_model
        
    def get_params(self, deep = False):
        return {'C': self.C,
                'eps': self.eps}


def SVR_scorer(estimator, X, y):
    return stats.pearsonr(estimator.svr_model.predict(X), y)[0]


def SVRCV_trainer(X, Y, c_eps_vals =  [(0.1, 0.1), (0.1, 1), (0.1, 10),
                                       (1, 0.1), (1, 1), (1, 10),
                                       (10, 0.1), (10, 1), (10, 10)]):
    comp_scores = []
    for C, eps in c_eps_vals:
        scores = model_selection.cross_val_score(SVREstimator(C=C, eps=eps), X, Y, cv=5, scoring=SVR_scorer, n_jobs=5)
        comp_scores.append(np.nanmean(scores))
    
    optimal_n_components_idx = comp_scores.index(max(comp_scores))
    opt_C, opt_eps = c_eps_vals[optimal_n_components_idx]
    svr_model = svm.LinearSVR(C=opt_C, epsilon=opt_eps).fit(X, Y)
    return svr_model
    
    
def SVRCV_predictor(X, model):
    return model.predict(X)

def SVRCV_cross_validate_fn(sess1_X, sess2_X, Y, tqdm_desc): 
    return cross_validate(sess1_X, sess2_X, Y,
                          SVRCV_trainer, SVRCV_predictor, default_scorer, 
                          center_X=False, scale_X=False, center_Y=True, scale_Y=True, 
                          folds=folds, enable_tqdm=False, verbose=False, tqdm_desc=tqdm_desc)

NonLinSVRCV_results = multi_cross_validate(ALL_PHEN, SVRCV_cross_validate_fn, n_procs=1)

In [ ]:
pickle.dump(NonLinSVRCV_results, open('./saved_models/SVRNonLin_results.pickle', 'wb'))

# Random Forest

In [ ]:
class RFEstimator:
    def __init__(self, max_depth, max_features, rand_state):
        self.max_depth = max_depth
        self.max_features = max_features
        self.rand_state = rand_state
        self.model = None

    def fit(self, X, y):
        model =  ensemble.RandomForestRegressor(n_estimators=5000, criterion='mse', max_depth=self.max_depth, max_features=self.max_features, random_state=self.rand_state, n_jobs=-1).fit(X, y)
        self.model = model
        
    def get_params(self, deep = False):
        return {'max_depth': self.max_depth,
                'max_features': self.max_features,
                'rand_state':self.rand_state}


def RF_scorer(estimator, X, y):
    return stats.pearsonr(estimator.model.predict(X).flatten(), y)[0]


def RFCV_trainer(X, Y, depth_features=[(3, 'sqrt'), (3, 0.01), (3, 0.1),
                                       (9, 'sqrt'), (9, 0.01), (9, 0.1),
                                       (27, 'sqrt'), (27, 0.01), (27, 0.1)]):
    comp_scores = []
    for rand_state, (depth, n_features) in enumerate(depth_features):
        scores = model_selection.cross_val_score(RFEstimator(max_depth=depth, max_features=n_features, rand_state=rand_state), X, Y, cv=5, scoring=RF_scorer, n_jobs=5)
        comp_scores.append(np.nanmean(scores))
    
    optimal_n_components_idx = comp_scores.index(max(comp_scores))
    opt_depth, opt_features = depth_features[optimal_n_components_idx]
    rf_model = ensemble.RandomForestRegressor(n_estimators=5000, criterion='mse', max_depth=opt_depth, max_features=opt_features, random_state=optimal_n_components_idx, n_jobs=25).fit(X, Y)
    return rf_model
    
    
def RFCV_predictor(X, model):
    return model.predict(X).flatten()

def RFCV_cross_validate_fn(sess1_X, sess2_X, Y, tqdm_desc): 
    return cross_validate(sess1_X, sess2_X, Y,
                          RFCV_trainer, RFCV_predictor, default_scorer, 
                          center_X=False, scale_X=False, center_Y=False, scale_Y=False, 
                          folds=folds, enable_tqdm=False, verbose=False, tqdm_desc=tqdm_desc)

RFCV_results = multi_cross_validate(ALL_PHEN, RFCV_cross_validate_fn, n_procs=1)

In [ ]:
pickle.dump(RFCV_results, open('./saved_models/RF_results.pickle', 'wb'))